<a href="https://colab.research.google.com/github/yunn333/AI/blob/main/HW0415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.引入老師的基礎四套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. 用 OpenAI API 使用

導入open AI、API金鑰及模型

In [ ]:
import os
from google.colab import userdata

In [ ]:
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

### 3.使用 AI Suite

下載 AISuite 並進行設定

In [ ]:
!pip install aisuite[all]

In [ ]:
import aisuite as ai

provider_planner = "groq"
model_planner = "gemma2-9b-it"

provider_writer = "groq"
model_writer = "gemma2-9b-it"

In [ ]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

###  4. 設定機器人的兩階段

### 🎯 任務說明：將「員瑛式思考產生器」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，生成一段超級正向的社群貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼這是一件超級幸運的事」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（員瑛式風格 + 第一人稱 + emoji + "完全是 Lucky Vicky 呀!" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

In [ ]:
system_planner = """
你是一位專業的減重顧問與心理教練，擅長從使用者的日常行為中找出進步與正向的潛力。根據使用者輸入的今天生活、飲食、運動等描述，請列出五個「其實你正在變好」的觀察點。請用台灣中文，條列式列出。
"""

system_writer = """
你是一位專業又溫暖的減重教練，擅長鼓勵使用者建立正向思維。請從使用者的五個進步理由中，挑出一個最值得鼓勵的點，用一段話肯定使用者的努力與方向。請用台灣中文自然語氣，不要用太浮誇的詞彙，也不用寫成 IG 貼文，請專業而溫暖地說明即可。
"""

In [ ]:
def fit_post(prompt):
    planning_prompt = f"使用者說：{prompt}。請找出五個『這樣其實有在變好』的觀察點。"
    insights = reply(system_planner, planning_prompt,
                     provider=provider_planner, model=model_planner)

    encouragement_prompt = f"這是我找到的觀察點：\n{insights}\n\n請選出其中一項，寫一段溫暖鼓勵的話。"
    encouragement = reply(system_writer, encouragement_prompt,
                          provider=provider_writer, model=model_writer)

    return insights, encouragement

### 5. 使用 Gradio

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 減肥潛力分析師（Planning AI）")
    gr.Markdown("請輸入你今天的飲食／運動／生活情況，我會告訴你：你其實哪裡已經在進步")

    user_input = gr.Textbox(label="今天的情況是…", placeholder="例如：早餐吃蛋餅，午餐便當沒有吃完，沒運動但有多走路。")

    btn = gr.Button("🔍 分析我的潛在進步")

    with gr.Row():
        out1 = gr.Textbox(label="✅ 五個你正在進步的觀察點", lines=7)
        out2 = gr.Textbox(label="💬 一段專業又溫暖的鼓勵", lines=4)

    btn.click(fit_post, inputs=[user_input], outputs=[out1, out2])

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://434a088d12c66dfb8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
